In [35]:
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
import openai

# For ChatGPT-4
openai.api_key = os.getenv("OPENAI_API_KEY")
# alternatively set LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = ""  # "true"

try:
    models = openai.models.list()
    print("API key is valid. Available models:", [model.id for model in models.data])
except openai.error.AuthenticationError as e:
    print("API key is invalid:", str(e))

openai_llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"))


API key is valid. Available models: ['whisper-1', 'dall-e-2', 'gpt-3.5-turbo-16k', 'tts-1-hd-1106', 'tts-1-hd', 'gpt-4-turbo-2024-04-09', 'gpt-4-0125-preview', 'gpt-4-turbo-preview', 'gpt-4-turbo', 'gpt-3.5-turbo-instruct-0914', 'gpt-3.5-turbo', 'gpt-3.5-turbo-instruct', 'text-embedding-3-small', 'tts-1', 'text-embedding-3-large', 'gpt-4-1106-preview', 'babbage-002', 'gpt-4o', 'gpt-3.5-turbo-0125', 'gpt-4o-2024-05-13', 'tts-1-1106', 'dall-e-3', 'gpt-4-0613', 'text-embedding-ada-002', 'gpt-4', 'davinci-002', 'gpt-3.5-turbo-1106']


In [5]:
def llama3_request(prompt):
    # This function should interact with your local Llama3 model.
    # Replace the below code with actual API or command-line interaction.
    import subprocess
    result = subprocess.run(['ollama', 'run', 'llama3', '--prompt', prompt], capture_output=True, text=True)
    return result.stdout


In [59]:
model = ChatOpenAI()
parser = StrOutputParser()

# Prompt templates
translation_system_template = """Given the following text in {source_language}, 
translate the word marked as <b>{word}</b> from {source_language} to {target_language}. Put into result only word translation without surraunding <b></b>"""

translation_prompt_template = ChatPromptTemplate.from_messages([
    ('system', translation_system_template),
    ('user', '{text}')
])
translation_chain = translation_prompt_template | model | parser


examples_system_template = """Given the following text in {source_language}, 
provide three examples in {source_language} of sentences using the word marked as <b>{word}</b> in his meaning in the context."""

examples_prompt_template = ChatPromptTemplate.from_messages([
    ('system', examples_system_template),
    ('user', '{text}')
])
examples_chain = examples_prompt_template | model | parser


explanation_system_template = """Given the following text in {source_language}, 
using only {source_language} explain the meaning of the word marked as <b>{word}</b>."""

explanation_prompt_template = ChatPromptTemplate.from_messages([
    ('system', explanation_system_template),
    ('user', '{text}')
])
explanation_chain = explanation_prompt_template | model | parser


# Function to interact with Llama3
def llama3_chain(prompt_template, **kwargs):
    prompt = prompt_template.format(**kwargs)
    response = llama3_request(prompt)
    return response


In [57]:
def get_translation_examples_and_explanation(text: str, word: str, source_language: str, target_language: str, model: str = 'gpt'):
    # Mark the specific word in the text
    marked_text = text.replace(word, f"<b>{word}</b>", 1)
    if model == 'gpt':
        translation = translation_chain.invoke({
            "text": marked_text, "word": word, "source_language": source_language, "target_language": target_language})
        examples = examples_chain.invoke({
            "text": marked_text, "word": word, "source_language": source_language, "target_language": target_language})
        explanation = explanation_chain.invoke({
            "text": marked_text, "word": word, "source_language": source_language, "target_language": target_language})
    elif model == 'llama3':
        translation = llama3_chain(translation_prompt_template, text=marked_text, word=word, source_language=source_language, target_language=target_language)
        # examples = llama3_chain(example_prompt_template, text=marked_text, word=word, source_language=source_language, target_language=target_language)
        # explanation = llama3_chain(explanation_prompt_template, text=marked_text, word=word, source_language=source_language)
    return translation, examples, explanation

def format_results_in_html(translation: str, examples: str, explanation: str) -> str:
    html_content = f"""
    <div>
        <h2>Translation</h2>
        <p>{translation}</p>
        <h2>Usage Examples</h2>
        <ul>
    """
    for example in examples.split("\n"):
        html_content += f"<li>{example}</li>"
    
    html_content += f"""
        </ul>
        <h2>Explanation</h2>
        <p>{explanation}</p>
    </div>
    """
    return f"{translation}\n{examples}\n{explanation}"


In [60]:
# Example usage
serbian_text = "Ljubav je najlepša stvar na svetu. Ljubav pokreće sve."
word_to_translate = "Ljubav"
source_language = "Serbian"
target_language = "English"

# Test with ChatGPT-4
translation_gpt, examples_gpt, explanation_gpt = get_translation_examples_and_explanation(serbian_text, word_to_translate, source_language, target_language, model='gpt')

# Test with Llama3
# translation_llama, examples_llama, explanation_llama = get_translation_examples_and_explanation(serbian_text, word_to_translate, source_language, target_language, model='llama3')

# Display results in HTML
html_result_gpt = format_results_in_html(translation_gpt, examples_gpt, explanation_gpt)
# html_result_llama = format_results_in_html(translation_llama, examples_llama, explanation_llama)

from IPython.display import display, HTML

print("ChatGPT-4 Results:")
# display(HTML(html_result_gpt))
print(html_result_gpt)

# print("Llama3 Results:")
# display(HTML(html_result_llama))


ChatGPT-4 Results:
Love
1. Duboko verujem u snagu <b>ljubavi</b> koja nas spaja i čini da sve probleme prevaziđemo zajedno.
2. Njihova veza je puna strasti i <b>ljubavi</b>, moguće je osetiti toplinu njihovih osmeha dok su zajedno.
3. <b>Ljubav</b> prema prirodi je ono što me inspiriše da očuvam ovaj predivan svet za buduće generacije.
<b>Ljubav</b> predstavlja snažan osećaj topline, privrženosti i pažnje prema nekome ili nečemu. To je osećaj koji nas pokreće, čini da se osećamo srećno i ispunjeno.
